# Setup

In [1]:
! pip install -q -U datasets
! pip install -q nltk
! pip install -q sentence-transformers
! pip install -q -U huggingface_hub
# ! pip install -q pandas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have 

In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict
from sentence_transformers import SentenceTransformer, CrossEncoder, util

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data preprocessing

In [ ]:
dataset = DatasetDict()
dataset["train"] = load_dataset("legacy107/qa_wikipedia", split="train[:80%]")
dataset["validation"] = load_dataset("legacy107/qa_wikipedia", split="validation[:80%]")
dataset["test"] = load_dataset("legacy107/qa_wikipedia", split="test[:80%]")
dataset

In [6]:
import nltk

nltk.download("punkt")

article_cache = {}


def chunk_text(row, chunk_size=100, overlap=0.20):
    global article_cache

    overlap_size = chunk_size * overlap

    if row["title"] in article_cache.keys():
        row["chunked_article"] = article_cache[row["title"]]
        return row

    sentences = nltk.sent_tokenize(row["article"])

    chunks = []
    text = sentences[0]

    if len(sentences) == 1:
        chunks.append(text)

    i = 1
    while i < len(sentences):
        text += " " + sentences[i]
        i += 1
        while (
            i < len(sentences)
            and len(nltk.word_tokenize(f"{text} {sentences[i]}")) <= chunk_size
        ):
            text += " " + sentences[i]
            i += 1

        text = (
            text.replace('"', '"')
            .replace("'", "'")
            .replace("\n\n\n", " ")
            .replace("\n\n", " ")
            .replace("\n", " ")
        )
        chunks.append(text)

        if i >= len(sentences):
            break

        j = i - 1
        text = sentences[j]
        while (
            j >= 0 and len(nltk.word_tokenize(f"{sentences[j]} {text}")) <= overlap_size
        ):
            text = sentences[j] + " " + text
            j -= 1

    row["chunked_article"] = chunks
    article_cache[row["title"]] = chunks

    return row

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
dataset = dataset.map(chunk_text, batched=False)

Map:   0%|          | 0/110970 [00:00<?, ? examples/s]

Map:   0%|          | 0/13833 [00:00<?, ? examples/s]

Map:   0%|          | 0/13873 [00:00<?, ? examples/s]

In [10]:
dataset.push_to_hub("legacy107/qa_wikipedia_chunked")

Pushing dataset shards to the dataset hub:   0%|          | 0/32 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
article_cache = {}


def get_article(row):
    global article_cache

    if row["title"] not in article_cache.keys():
        article_cache[row["title"]] = row["chunked_article"]


dataset.map(get_article, batched=False)

# Retrieve context

In [24]:
dataset = load_dataset("legacy107/qa_wikipedia_chunked")

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/110970 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13833 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13873 [00:00<?, ? examples/s]

In [25]:
bi_encoder = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")
bi_encoder.max_seq_length = 256  # Truncate long passages to 256 tokens
top_k = 10  # Number of passages we want to retrieve with the bi-encoder

# The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [29]:
chunked_articles_corpus = []
for chunk in dataset["train"][0]["chunked_article"]:
    chunked_articles_corpus.append(chunk)

corpus_embeddings = bi_encoder.encode(
    chunked_articles_corpus, convert_to_tensor=True, show_progress_bar=True
)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, chunked_articles_corpus[hit["corpus_id"]]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]["cross-score"] = cross_scores[idx]

    # Output of top-3 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x["score"], reverse=True)
    for hit in hits[0:3]:
        print(
            "\t{:.3f}\t{}".format(
                hit["score"],
                chunked_articles_corpus[hit["corpus_id"]].replace("\n", " "),
            )
        )

    # Output of top-3 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x["cross-score"], reverse=True)
    for hit in hits[0:3]:
        print(
            "\t{:.3f}\t{}".format(
                hit["cross-score"],
                chunked_articles_corpus[hit["corpus_id"]].replace("\n", " "),
            )
        )

In [31]:
print(dataset["train"][0]["question"])
print(dataset["train"][0]["answer"])
print(dataset["train"][0]["context"])
print(search(query=dataset["train"][0]["question"]))

what is congressional oversight intended to do ?
prevent waste and fraud
Congressional oversight is intended to prevent waste and fraud, protect civil liberties and individual rights, ensure executive compliance with the law, gather information for making laws and educating the public, and evaluate executive performance.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.793	These disputes have often been the subject of lawsuits that have ultimately been decided by the United States Supreme Court. ==== Congressional oversight ==== Congressional oversight is intended to prevent waste and fraud, protect civil liberties and individual rights, ensure executive compliance with the law, gather information for making laws and educating the public, and evaluate executive performance.It applies to cabinet departments, executive agencies, regulatory commissions, and the presidency.
	0.788	==== Congressional oversight ==== Congressional oversight is intended to prevent waste and fraud, protect civil liberties and individual rights, ensure executive compliance with the law, gather information for making laws and educating the public, and evaluate executive performance.It applies to cabinet departments, executive agencies, regulatory commissions, and the presidency. Congress' oversight function takes ma

In [32]:
embedding_cache = {}


def retrieve_context(row):
    global embedding_cache

    if row["title"] in embedding_cache.keys():
        corpus_embeddings = embedding_cache[row["title"]]
    else:
        corpus_embeddings = bi_encoder.encode(
            row["chunked_article"], convert_to_tensor=True, show_progress_bar=False
        )
        embedding_cache[row["title"]] = corpus_embeddings

    query = row["question"]

    question_embedding = bi_encoder.encode(
        query, convert_to_tensor=True, show_progress_bar=False
    )
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]

    cross_inp = [[query, row["chunked_article"][hit["corpus_id"]]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp, show_progress_bar=False)

    for idx in range(len(cross_scores)):
        hits[idx]["cross-score"] = cross_scores[idx]

    hits = sorted(hits, key=lambda x: x["cross-score"], reverse=True)

    row["retrieved_context"] = " ".join(
        [row["chunked_article"][hit["corpus_id"]] for hit in hits[0:3]]
    ).replace("\n", " ")

    return row

In [33]:
dataset = dataset.map(retrieve_context, batched=False)

Map:   0%|          | 0/110970 [00:00<?, ? examples/s]

Map:   0%|          | 0/13833 [00:00<?, ? examples/s]

Map:   0%|          | 0/13873 [00:00<?, ? examples/s]

In [9]:
dataset["validation"].filter(lambda x: len(x["chunked_article"]) == 0)

Filter:   0%|          | 0/13833 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answer_start', 'answer', 'article', 'chunked_article'],
    num_rows: 0
})

In [42]:
chunk_text(dataset["validation"].filter(lambda x: len(x["chunked_article"]) == 0)[0])

['Telephones – main lines in use:\n47,000 (1995)\nTelephones – mobile cellular:\nNA\nTelephone system:\ndomestic:\nfair open wire and microwave radio relay system\ninternational:\nsatellite earth station – 1 Intelsat (Atlantic Ocean)\nRadio stations:\nAM 2, FM 14 (including 6 repeaters), shortwave 6 (including 5 repeaters) (1998)\nRadios:\n104,000 (1997)\nTelevision stations:\n3 (plus eight low-power repeaters) (1997)\nTelevisions:\n30,000 (1997)\nInternet Service Providers (ISPs):\nNA\nCountry code (Top-level domain): GF\n\n\n== See also ==\nFrench Guiana\nTelecommunications in France\n\n\n== External links ==\nInterreg Caribbean\nFrench Guiana, SubmarineCableMap.com']


{'id': '152268',
 'title': 'Telecommunications in French Guiana',
 'context': 'Televisions:\n30,000 (1997)',
 'question': 'when was televisions televisions ?',
 'answer_start': 13,
 'answer': '30,000 (1997)',
 'article': 'Telephones – main lines in use:\n47,000 (1995)\nTelephones – mobile cellular:\nNA\nTelephone system:\ndomestic:\nfair open wire and microwave radio relay system\ninternational:\nsatellite earth station – 1 Intelsat (Atlantic Ocean)\nRadio stations:\nAM 2, FM 14 (including 6 repeaters), shortwave 6 (including 5 repeaters) (1998)\nRadios:\n104,000 (1997)\nTelevision stations:\n3 (plus eight low-power repeaters) (1997)\nTelevisions:\n30,000 (1997)\nInternet Service Providers (ISPs):\nNA\nCountry code (Top-level domain): GF\n\n\n== See also ==\nFrench Guiana\nTelecommunications in France\n\n\n== External links ==\nInterreg Caribbean\nFrench Guiana, SubmarineCableMap.com',
 'chunked_article': []}

In [34]:
dataset.remove_columns("chunked_article").push_to_hub(
    "legacy107/qa_wikipedia_retrieved_chunks-og"
)

Pushing dataset shards to the dataset hub:   0%|          | 0/13 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

In [4]:
dataset = load_dataset("legacy107/qa_wikipedia_retrieved_chunks")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/110970 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13833 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13873 [00:00<?, ? examples/s]

In [35]:
match = 0


def find_match(row):
    global match
    match = match + (1 if row["answer"] in row["retrieved_context"] else 0)


dataset.map(find_match, batched=False)

print("%.2f" % (match / sum(dataset.num_rows.values()) * 100))

Map:   0%|          | 0/110970 [00:00<?, ? examples/s]

Map:   0%|          | 0/13833 [00:00<?, ? examples/s]

Map:   0%|          | 0/13873 [00:00<?, ? examples/s]

87.10


In [ ]:
fine-tuned: 39.04